<a href="https://colab.research.google.com/github/ykitaguchi77/FundusPhoto/blob/main/Preprocess_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Preprocess_images**

In [5]:
from PIL import Image
import os
import time
from IPython.display import display
import numpy as np
import time
import os
import copy
import math
import shutil
import csv
import pandas as pd
import glob
from PIL import Image


#google driveをcolabolatoryにマウント
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#**224pxに縮小して保存（Image.BILINEAR）**

In [3]:
path = '/content/drive/MyDrive/Deep_learning/FundusPhoto'
ext = 'png'  #png or jpg

os.chdir(path)


#元画像フォルダ
in_path = ['cropped_2_img', 'macula_2_img', 'vascular_2_img']

#保存先フォルダ
out_path = ['cropped_224_img', 'macula_224_img', 'vascular_224_img']


In [4]:
start_time = time.time()
process = [0,1,2]

for i in process:
    #保存先フォルダの作成
    if os.path.exists(out_path[i]):
        shutil.rmtree(out_path[i])
    os.makedirs(out_path[i])

    k=0
    for j in os.listdir(in_path[i]):
        path = os.path.join(in_path[i], str(j))
        print(path)
        pilr_img = Image.open(path).convert('RGB')
        converted_img = pilr_img.resize((224, 224), Image.BILINEAR)
        converted_img.save(os.path.join(out_path[i], str(j)))
        print(str(k)+' images converted!')
        k+=1
    
    
    # calculate elapsed time
    elapsed_time = int(time.time() - start_time)

    # convert second to hour, minute and seconds
    elapsed_hour = elapsed_time // 3600
    elapsed_minute = (elapsed_time % 3600) // 60
    elapsed_second = (elapsed_time % 3600 % 60)

    # print as 00:00:00
    print(str(elapsed_hour).zfill(2) + ":" + str(elapsed_minute).zfill(2) + ":" + str(elapsed_second).zfill(2))




0 images converted!
1 images converted!
2 images converted!
3 images converted!
4 images converted!
5 images converted!
6 images converted!
7 images converted!
8 images converted!
9 images converted!
10 images converted!
11 images converted!
12 images converted!
13 images converted!
14 images converted!
15 images converted!
16 images converted!
17 images converted!
18 images converted!
19 images converted!
20 images converted!
21 images converted!
22 images converted!
23 images converted!
24 images converted!
25 images converted!
26 images converted!
27 images converted!
28 images converted!
29 images converted!
30 images converted!
31 images converted!
32 images converted!
33 images converted!


KeyboardInterrupt: ignored